In [1]:
import tensorflow as tf
from tensorflow import keras

# 加载TF模块
from tensorflow.keras import layers,Sequential,losses,optimizers,datasets
from tensorflow.keras.datasets import mnist

# 10. 卷积神经网络

## 10.1 全连接网络的问题
首先我们来分析全连接网络存在的问题。考虑一个简单的4层全连接层网络，如`图10.1`所示。

<img src="images/10_01.png" style="width:600px;"/>

通过`TensorFlow`快速地搭建此网络模型：

In [2]:
# 创建4层全连接网络
model = keras.Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10),
])

# build模型，并打印模型信息
model.build(input_shape=(4, 784))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  200960    
_________________________________________________________________
dense_1 (Dense)              multiple                  65792     
_________________________________________________________________
dense_2 (Dense)              multiple                  65792     
_________________________________________________________________
dense_3 (Dense)              multiple                  2570      
Total params: 335,114
Trainable params: 335,114
Non-trainable params: 0
_________________________________________________________________


`summary()`函数打印出模型每一层的参数量统计结果。

网络的参数量是怎么计算的呢？对于每一条连接线的权值标量，视作一个参数，因此对输入节点数为$n$，输出节点数为$m$的全连接层来说，$W$张量包含的参数量共有$n \cdot m$个，$b$向量包含的参数量有$m$个，则全连接层的总参数量为$n\cdot m+m$。上面的四层网络总参数量约34万个。如果将单个权值保存为`float`类型的变量，至少需要占用4个字节内存(`Python`语言中`float`占用内存更多)，那么34万个网络参数至少需要约1.34MB内存。网络的训练过程中还需要缓存计算图模型、梯度信息、输入和中间计算结果等，其中梯度相关运算占用资源非常多。

那么训练这样一个网络到底需要多少内存呢？我们可以在现代GPU 备上简单模拟一下资源消耗情况。在`TensorFlow`中，如果不设置显存占用方式，那么默认会占用全部显存。这里将`TensorFlow`的显存使用方式设置为按需分配：

```python
# 获取所有 GPU 设备列表
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # 设置 GPU 显存占用为按需分配，增长式
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # 异常处理
        print(e)
```

设置`TensorFlow`按需申请显存资源，这样`TensorFlow`占用的显存大小即为运算需要的数量。在`Batch Size`设置为32的情况下，我们观察到显存占用了约708MB，内存占用约870MB。

### 10.1.1 局部相关性
接下来我们探索如何避免全连接网络的参数量过大的缺陷。为了便于讨论，我们以图片类型数据为输入的场景为例。对于2D的图片数据，在进入全连接层之前，需要将矩阵数据打平成1D向量，然后每个像素点与每个输出节点两两相连，如`图10.2`所示。

<img src="images/10_02.png" style="width:500px;"/>

网络层的每个输出节点与所有的输入节点相连接，用于提取所有输入节点的特征信息，这种稠密的连接方式是全连接层参数量大、计算代价高的根本原因。全连接层也称为`稠密连接层`(Dense Layer)，输出与输入的关系为：
+ $o_j = \sigma\bigg(\displaystyle\sum_{i \in nodes(I)}w_{ij}x_i+b_j\bigg)$

其中$nodes(I)$表示$I$层的节点集合。

那么，输出节点是否有必要和全部的输入节点相连接呢？我们可以分析输入节点对输出节点的重要性分布，仅考虑较重要的一部分输入节点， 而抛弃重要性较低的部分节点，这样输出节点只需要与部分输入节点相连接，表达为：
+ $o_j = \sigma\bigg(\displaystyle\sum_{i \in top(I,j,k)}w_{ij}x_i+b_j\bigg)$

其中$top(I,j,k)$表示$I$层中对于$J$层中的$j$号节点重要性最高的前$k$个节点集合。通过这种方式，可以把全连接层的$\lVert I\rVert\cdot\lVert J\rVert$个权值连接减少到$k\cdot\lVert J\rVert$个，其中$\lVert I\rVert$、$\lVert J\rVert$分布表示$I,J$层的节点数量。

现在问题就转变为探索$I$层输入节点对于$j$号输出节点的重要性分布。找出每个中间节点的重要性分布是件非常困难的事情，我们可以针对于具体问题，利用先验知识把这个问题进一步简化。

在现实生活中，存在着大量以位置或距离作为重要性分布衡量标准的数据，比如股票的走势预测应该更加关注近段时间的数据趋势(时间相关)、图片每个像素点和周边像素点的关联度更大(位置相关)。

以2D图片数据为例，如果简单地认为与当前像素欧式距离(Euclidean Distance)小于和等于$\displaystyle\frac{k}{\sqrt{2}}$的像素点重要性较高，反之则重要性较低，那么我们就很轻松地简化了每个像素点的重要性分布问题。如`图10.3`所示

<img src="images/10_03.png" style="width:300px;"/>

这个高宽为$k$的窗口称为`感受野`(Receptive Field)，它表征了每个像素对于中心像素的重要性分布情况，网格内的像素才会被考虑。

这种基于距离的重要性分布假设特性称为`局部相关性`。在这种重要性分布假设下，全连接层的连接模式变成了如`图10.4`：

<img src="images/10_04.png" style="width:300px;"/>

利用局部相关性的思想，我们把感受野窗口的高、宽记为$k$(实际上感受野的高、宽可以不相等)，当前位置的节点与大小为$k$的窗口内的所有像素相连接，网络层的输入输出关系表达如下：
+ $o_j = \sigma\bigg(\displaystyle\sum_{\text{dist}(i,j) \le \frac{k}{\sqrt{2}}}w_{ij}x_i+b_j\bigg)$

### 10.1.2 权值共享
每个输出节点仅与感受野区域内$k\times k$个输入节点相连接，输出层节点数为$\lVert J\rVert$，则当前层的参数量为$k \times k \times \lVert J\rVert$，相对于全连接层的$\lVert I\rVert\times\lVert J\rVert$，考虑到$k$(如1、3、5)，网络模型参数量减少了很多。

能否再将参数量进一步减少，比如只需要$k\times k$个参数即可完成当前层的计算？答案是肯定的，通过`权值共享`的思想，对于每个输出节点$o_j$，均使用相同的权值矩阵$W$，那么无论输出节点的数量$\lVert J\rVert$是多少，网络层的参数量总是$k\times k$。如`图10.5`所示，在计算左上角 位置的输出像素时，使用权值矩阵：
+ $W = \begin{bmatrix}w_{11}&w_{12}&w_{13}\\w_{21}&w_{22}&w_{23}\\w_{31}&w_{32}&w_{33}\\ \end{bmatrix}$

与对应感受野内部的像素相乘累加，作为左上角像素的输出值；在计算右下方感受野区域时，使用相同的权值参数$W$相乘累加，得到右下角像素的输出值，此时网络层的参数量只有$3 \times 3 = 9$个，且与输入、输出节点数无关。

<img src="images/10_05.png" style="width:300px;"/>

通过运用`局部相关性`和`权值共享`的思想，我们把网络的参数量减少到$k \times k$(准确地说，是在单输入通道、单卷积核的条件下)。这种共享权值的`局部连接层`网络其实就是卷积神经网络。

## 10.2 卷积神经网络
卷积神经网络通过充分利用局部相关性和权值共享的思想，大大地减少了网络的参数量，从而提高训练效率，更容易实现超大规模的深层网络。

本节介绍卷积神经网络层的具体计算流程。以2D图片数据为例，卷积层接受高、宽分别为$h、w$，通道数为$c_{in}$的输入特征图$X$，在$c_{out}$个高、宽都为$k$，通道数为$c_{in}$的 卷积核作用下，生成高、宽分别为$h'、w'$，通道数为$c_{out}$的特征图输出。需要注意的是，卷积核的高宽可以不等。

我们首先从单通道输入、单卷积核的情况开始讨论，然后推广至多通道输入、单卷积核，最后讨论最常用，也是最复杂的多通道输入、多个卷积核的卷积层实现。

### 10.2.1 单通道输入和单卷积核
首先讨论单通道输入$c_{in} = 1$，如灰度图片只有灰度值一个通道，单个卷积核$c_{out} = 1$的情况。以输入$X$为$5 \times 5$的矩阵，卷积核为$3 \times 3$的矩阵为例，如`图10.12`所示。

<img src="images/10_12.png" style="width:500px;"/>

首先移动至输入$X$最左上方(上方的绿色方框)，选中输入$X$上$3 \times 3$的感受野元素，与卷积核(图片中间$3 \times 3$方框)对应元素相乘：
+ $\begin{bmatrix}1&-1&0\\-1&-2&2\\1&2&-2\\ \end{bmatrix} \odot \begin{bmatrix}-1&1&2\\1&-1&3\\0&-1&-2\\ \end{bmatrix} = \begin{bmatrix}-1&-1&0\\-1&2&6\\0&-2&4\\ \end{bmatrix}$

$\odot$表示哈达马积(Hadamard Product)，即矩阵的对应元素相乘，它与矩阵相乘符号@是矩阵的二种最为常见的运算形式。运算后得到$3\times 3$的矩阵，这9个数值全部相加：$−1−1+0−1+2+6+0−2+4=7$，得到标量7，写入输出矩阵第一行、第一列的位置。

完成第一个感受野区域的特征提取后，感受野窗口向右移动一个步长单位(Strides，默认为1)，选中`图10.13`中绿色方框标示的感受野元素，按照同样的计算方法得到输出10，写入第一行、第二列位置。

<img src="images/10_13.png" style="width:500px;"/>

感受野窗口再次向右移动一个步长单位计算输出，如`图10.14`所示。

<img src="images/10_14.png" style="width:500px;"/>

此时感受野已经移动至输入$X$的有效像素的最右边，此时，感受野窗口向下移动一个步长单位并回到当前行的行首位置，继续选中新的感受野元素区域，如`图10.15`所示。

<img src="images/10_15.png" style="width:500px;"/>

按照上述方法，每次感受野向右移动1个步长单位，若超出输入边界，则向下移动1个步长单位，并回到行首，直到感受野移动至最右边、最下方位置，如下`图10.16`所示。

<img src="images/10_16.png" style="width:500px;"/>

### 10.2.2 多通道输入和单卷积核
多通道输入的卷积层更为常见，比如彩色的图片包含了`R/G/B`三个通道，每个通道上面的像素值表示`R/G/B`色彩的强度。下面我们以3通道输入、单个卷积核为例，将单通道输入的卷积运算方法推广到多通道的情况。

如`图10.17`中所示，每行的最左边$5\times5$的矩阵表示输入$X$的`1~3`通道，第2列的$3\times3$矩阵分别表示卷积核的`1~3`通道，第3列的矩阵表示当前通道上运算结果的中间矩阵，最右边一个矩阵表示卷积层运算的最终输出。

<img src="images/10_17.png" style="width:500px;"/>

在多通道输入的情况下，卷积核的通道数需要和输入$X$的通道数量相匹配，卷积核的第$i$个通道和$X$的第$i$个通道运算，得到第$i$个中间矩阵，此时可以视为单通道输入与单卷积核的情况，所有通道的中间矩阵对应元素再次相加，作为最终输出。

具体的计算流程如下：在初始状态，如`图10.17`所示，每个通道上面的感受野窗口同步落在对应通道上面的最左边、最上方位置，每个通道上感受野区域元素与卷积核对应通道上面的矩阵相乘累加，分别得到三个通道上面的输出`7、-11、-1`的中间变量，这些中间变量相加得到输出-5，写入对应位置。

随后，感受野窗口同步在$X$的每个通道上向右移动1个步长单位，如`图10.18`所示，每个通道上面的感受野与卷积核对应通道上面的矩阵相乘累加，得到中间变量`10、20、20`，全部相加得到输出50，写入第一行、第二列元素位置。

<img src="images/10_18.png" style="width:500px;"/>

以此方式同步移动感受野窗口，直至最右边、最下方位置，此时全部完成输入和卷积核的卷积运算，得到$3\times3$的输出矩阵，如`图10.19`所示。

<img src="images/10_19.png" style="width:500px;"/>

整个计算如`图10.20`所示，输入的每个通道处的感受野均与卷积核的对应通道相乘累加，得到与通道数量相等的中间变量，这些中间变量全部相加即得到当前位置的输出值。输入通道的通道数量决定了卷积核的通道数。一个卷积核只能得到一个输出矩阵，无论输入$X$的通道数量。

<img src="images/10_20.png" style="width:300px;"/>

一般来说，一个卷积核只能完成某种逻辑的特征提取，当需要同时提取多种逻辑特征时，可以通过增加多个卷积核来得到多种特征，提高神经网络的表达能力，这就是多通道输入、多卷积核的情况。

### 10.2.3 多通道输入、多卷积核
多通道输入、多卷积核是卷积神经网络中最为常见的形式。

当出现多卷积核时，第$i$($i \in n$，$n$为卷积核个数)个卷积核与输入$X$运算得到第$i$个输出矩阵(也称为输出张量$O$的通道$i$)，最后全部的输出矩阵在通道维度上进行拼接(Stack操作，创建输出通道数的新维度)，产生输出张量$O$，$O$包含了$n$个通道数。

以3通道输入、2个卷积核的卷积层为例。第一个卷积核与输入$X$运算得到输出$O$的第一个通道，第二个卷积核与输入$X$运算得到输出$O$的第二个通道，如`图10.21`所示，输出的两个通道拼接在一起形成了最终输出$O$。每个卷积核的大小$k$、步长$s$、填充设定等都是统一设置，这样才能保证输出的每个通道大小一致，从而满足拼接的条件。

<img src="images/10_21.png" style="width:350px;"/>

### 10.2.4 步长
在卷积运算中，如何控制感受野布置的密度呢？
+ 对于信息密度较大的输入，如物体数量很多的图片，为了尽可能的少漏掉有用信息，在网络设计的时候希望能够较密集地布置感受野窗口
+ 对于信息密度较小的输入，比如全是海洋的图片，可以适量的减少感受野窗口的数量

感受野密度的控制手段一般是通过移动步长(Strides)实现的。`图10.22`中感受野沿$x$方向的步长为：

<img src="images/10_22.png" style="width:500px;"/>

当感受野移动至输入$X$右边的边界时，感受野向下移动一个步长$s = 2$，并回到行首：

<img src="images/10_23.png" style="width:500px;"/>

循环往复移动，直至达到最下方、最右边边缘位置，如`图10.24`所示，最终卷积层输出的高宽只有$2\times2$，输出高宽由$3\times3$降低为$2\times2$，感受野的数量减少为4个。

<img src="images/10_24.png" style="width:500px;"/>

可以看到，通过设定步长$s$，可以有效地控制信息密度的提取：
+ 当步长设计的较小时，感受野以较小幅度移动窗口，有利于提取到更多的特征信息，输出张量的尺寸也更大
+ 当步长设计的较大时，感受野以较大幅度移动窗口，有利于减少计算代价，过滤冗余信息，输出张量的尺寸也更小

### 10.2.5 填充
经过卷积运算后的输出$O$的高宽一般会小于输入$X$的高宽。在网络模型设计时，有时希望输出$O$的高宽能够与输入$X$的高宽相同，从而方便网络参数的设计、残差连接等。一般的做法是通过在原输入$X$的高和宽维度上面进行填充(Padding)若干无效元素操作，得到增大的输入$X'$。通过精心设计填充单元的数量，在$X'$上面进行卷积运算得到输出$O$的高宽可以和原输入$X$相等，甚至更大。

`图10.25`中通过填充得到新的输入`𝑿'`：

<img src="images/10_25.png" style="width:500px;"/>

填充后的输入通过$3\times3$的卷积核得到$5\times5$的输出：

<img src="images/10_28.png" style="width:500px;"/>


## 10.3 卷积层实现
在`TensorFlow`中，既可以通过自定义权值的底层实现方式搭建神经网络，也可以直接调用现成的卷积层类的高层方式快速搭建复杂网络。

### 10.3.1 自定义权值
在`TensorFlow`中，通过`tf.nn.conv2d`函数可以方便地实现2D卷积运算。`tf.nn.conv2d`基于输入$X:[b,h,w,c_{in}]$和卷积核$W:[k,k,c_{in},c_{out}]$进行卷积运算，得到输出$O:[b,h',w',c_{out}]$，其中$c_{in}$表示输入通道数，$c_{out}$表示卷积核的数量，也是输出特征图的通道数。

例如：

In [3]:
# 模拟输入，3通道，高宽为5
x = tf.random.normal([2,5,5,3]) 
# 需要根据[k,k,cin,cout]格式创建W张量
w = tf.random.normal([3,3,3,4])
# 步长为1, padding为0,
out = tf.nn.conv2d(x,w,strides=1,padding=[[0,0],[0,0],[0,0],[0,0]])
out.shape

TensorShape([2, 3, 3, 4])

设置一下`padding`，`padding`参数的设置格式为：`padding=[[0,0],[上,下],[左,右],[0,0]]`：

In [4]:
# 模拟输入，3通道，高宽为5
x = tf.random.normal([2,5,5,3]) 
# 需要根据[k,k,cin,cout]格式创建W张量
w = tf.random.normal([3,3,3,4])
out = tf.nn.conv2d(x,w,strides=1,padding=[[0,0],[1,1],[1,1],[0,0]])
out.shape

TensorShape([2, 5, 5, 4])

特别地，通过设置参数`padding='SAME'、strides=1`可以直接得到输入、输出同大小的卷积层，`padding`的具体数量由`TensorFlow`自动计算并完成填充操作：

In [5]:
# 模拟输入，3通道，高宽为5
x = tf.random.normal([2,5,5,3]) 
# 需要根据[k,k,cin,cout]格式创建W张量
w = tf.random.normal([3,3,3,4])
out = tf.nn.conv2d(x,w,strides=1,padding='SAME')
out.shape

TensorShape([2, 5, 5, 4])

当$s\gt1$时，设置`padding='SAME'`将使得输出高、宽将成$\displaystyle\frac{1}{s}$倍地减少：

In [6]:
# 模拟输入，3通道，高宽为5
x = tf.random.normal([2,5,5,3]) 
# 需要根据[k,k,cin,cout]格式创建W张量
w = tf.random.normal([3,3,3,4])
# 高宽先padding成可以整除3的最小整数6，然后6按3倍减少，得到2x2
out = tf.nn.conv2d(x,w,strides=3,padding='SAME')
out.shape

TensorShape([2, 2, 2, 4])

卷积神经网络层也可以设置网络带偏置向量。`tf.nn.conv2d`函数是没有实现偏置向量计算的，添加偏置需要手动累加偏置张量：

In [7]:
# 根据[cout]格式创建偏置向量
b = tf.zeros([4])
# 在卷积输出上叠加偏置向量，它会自动broadcasting为[b,h',w',cout]
out = out + b

### 10.3.2 卷积层类
通过卷积层类`layers.Conv2D`可以不需要手动定义卷积核$W$和偏置$b$张量，直接调用类实例即可完成卷积层的前向计算。

> 在`TensorFlow`中，首字母大写的对象一般表示类，全部小写的一般表示函数，如`layers.Conv2D`表示卷积层类，`nn.conv2d`表示卷积运算函数

在新建卷积层类时，需要指定卷积核数量参数`filters`，卷积核大小`kernel_size`，步长`strides`，填充`padding`等：

In [8]:
layer = layers.Conv2D(4,kernel_size=3,strides=1,padding='SAME')

如果卷积核高宽不等，步长行列方向不等，参数需要使用`tuple`类型指定：

In [9]:
layer = layers.Conv2D(4,kernel_size=(3,4),strides=(2,1),padding='SAME')

创建完成后，通过调用实例(的`__call__`方法)即可完成前向计算：

In [10]:
layer = layers.Conv2D(4,kernel_size=3,strides=1,padding='SAME')
out = layer(x) 
out.shape 

TensorShape([2, 5, 5, 4])

在类`Conv2D`中，保存了卷积核张量$W$和偏置$b$，可以通过类成员`trainable_variables`访问：

In [11]:
for v in layer.trainable_variables:
    print(v.name, v.shape, v.dtype)

conv2d_2/kernel:0 (3, 3, 3, 4) <dtype: 'float32'>
conv2d_2/bias:0 (4,) <dtype: 'float32'>


也可以直接调用类实例`layer.kernel`、`layer.bias`访问$W$和$b$张量：

In [12]:
layer.kernel.name, layer.kernel.shape

('conv2d_2/kernel:0', TensorShape([3, 3, 3, 4]))

## 10.4 LeNet-5 实战
1990年代，`Yann LeCun`等人提出了用于手写数字和机器打印字符图片识别的神经网络，被命名为`LeNet-5`，网络结构如`图10.30`：

<img src="images/10_30.png" style="width:600px;"/>

它接受$32\times32$大小的数字、字符图片，经过第一个卷积层得到$28\times28$形状的张量，经过一个向下采样层，张量尺寸缩小到$[b,14,14,6]$，经过第二个卷积层，得到形状$[b,10,10,16]$的张量，同样经过下采样层，张量尺寸缩小到$[b,5,5,16]$，在进入全连接层之前，先将张量打成$[b,400]$的张量，送入输出节点数分别为120、84的2个全连接层，得到$[b,84]$的张量，最后通过`Gaussian connections`层。

我们在`LeNet-5`的基础上进行了少许调整，使得它更容易在现代深度学习框架上实现。首先我们将输入$X$形状调整为$28\times28$，然后将2个下采样层实现为`最大池化层`，最后利用`全连接层`替换掉`Gaussian connections层`。下文统一称修改的网络也为`LeNet-5`网络：

<img src="images/10_31.png" style="width:500px;"/>

我们基于`MNIST`手写数字图片数据集训练`LeNet-5`网络：

In [13]:
network = Sequential([
    layers.Conv2D(6,kernel_size=3,strides=1), # 第一个卷积层, 6个3x3卷积核
    layers.MaxPooling2D(pool_size=2,strides=2), # 高宽各减半的池化层
    layers.ReLU(), # 激活函数
    layers.Conv2D(16,kernel_size=3,strides=1), # 第二个卷积层, 16个3x3卷积核
    layers.MaxPooling2D(pool_size=2,strides=2), # 高宽各减半的池化层
    layers.ReLU(), # 激活函数
    layers.Flatten(), # 打平层，方便全连接层处理
    layers.Dense(120, activation='relu'), # 全连接层，120个节点
    layers.Dense(84, activation='relu'), # 全连接层，84个节点
    layers.Dense(10) # 全连接层，10个节点
])

# build一次网络模型，给输入X的形状，其中4为随意给的 batchsz
network.build(input_shape=(4, 28, 28, 1))
network.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            multiple                  60        
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
conv2d_4 (Conv2D)            multiple                  880       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
re_lu_1 (ReLU)               multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                 

卷积层的参数量非常少，主要的参数量集中在全连接层，而`10.1节`的全连接网络参数量达到了34万个，因此通过卷积神经网络可以显著降低网络参数量，同时增加网络深度。

在训练阶段，首先将输入$X$增加一个维度，即调整`shape`为$[b,28,28,1]$，送入模型进行前向计算，得到输出张量`output`的`shape`为$[b,10]$。我们新建`交叉熵损失函数`类来定义损失函数：

In [14]:
# 创建损失函数的类，在实际计算时直接调用类实例即可
# 通过设定`from_logits=True`将`softmax`激活函数实现在损失函数中，
# 不需要手动添加损失函数，提升数值计算稳定性
criteon = losses.CategoricalCrossentropy(from_logits=True)

In [26]:
(xs, ys), (xt, yt) = mnist.load_data()
xs = tf.convert_to_tensor(xs, dtype=tf.float32) / 255.
db = tf.data.Dataset.from_tensor_slices((xs, ys))
db = db.batch(32).repeat(10)

xt = tf.convert_to_tensor(xt, dtype=tf.float32) / 255.
dbtest = tf.data.Dataset.from_tensor_slices((xt, yt))
dbtest = dbtest.batch(32)

optimizer = optimizers.SGD(lr=0.01)
# acc_meter = metrics.Accuracy()

for step, (x,y) in enumerate(db):
    # 构建梯度记录环境
    with tf.GradientTape() as tape: 
        # 插入通道维度，=>[b,28,28,1]
        x = tf.expand_dims(x,axis=3)
        # 前向计算，获得10类别的预测分布，[b, 784] => [b, 10]
        out = network(x)
        # 真实标签one-hot编码，[b] => [b, 10]
        y_onehot = tf.one_hot(y, depth=10)
        # 计算交叉熵损失函数，标量
        loss = criteon(y_onehot, out)
    # 自动计算梯度
    grads = tape.gradient(loss, network.trainable_variables)
    # 自动更新参数
    optimizer.apply_gradients(zip(grads, network.trainable_variables))
    # 打印状态
    if step % 1000 == 0:
        print("#", step, ', loss: ', float(loss))

# 0 , loss:  0.04207484424114227
# 1000 , loss:  0.15341603755950928
# 2000 , loss:  0.042185381054878235
# 3000 , loss:  0.03214021399617195
# 4000 , loss:  0.07692582905292511
# 5000 , loss:  0.021433787420392036
# 6000 , loss:  0.026960991322994232
# 7000 , loss:  0.0038292957469820976
# 8000 , loss:  0.024503866210579872
# 9000 , loss:  0.06588927656412125
# 10000 , loss:  0.043183520436286926
# 11000 , loss:  0.016418201848864555
# 12000 , loss:  0.0035218519624322653
# 13000 , loss:  0.021439535543322563
# 14000 , loss:  0.09355004131793976
# 15000 , loss:  0.011912934482097626
# 16000 , loss:  0.046345874667167664
# 17000 , loss:  0.026961183175444603
# 18000 , loss:  0.027369201183319092


测试阶段不需要记录梯度信息，因此不需要`tf.GradientTape() as tape`环境。前向计算得到的输出经过`softmax`函数后，代表了网络预测当前图片属于类别$i$的概率。通过`argmax`函数选取概率最大的元素所在的索引，作为当前图片的预测类别：

In [27]:
correct, total = 0,0
for x,y in dbtest: # 遍历所有训练集样本
    # 插入通道维度，=>[b,28,28,1]
    x = tf.expand_dims(x,axis=3)
    # 前向计算，获得10类别的预测分布，[b, 784] => [b, 10]
    out = network(x)
    # 真实的流程时先经过softmax，再argmax
    # 但是由于softmax不改变元素的大小相对关系，故省去
    pred = tf.argmax(out, axis=-1)  
    y = tf.cast(y, tf.int64)
    # 统计预测正确数量
    correct += float(tf.reduce_sum(tf.cast(tf.equal(pred, y),tf.float32)))
    # 统计预测样本总数
    total += x.shape[0]
# 计算准确率
print('test acc:', correct/total)

test acc: 0.9837


在数据集上面循环训练30个`Epoch`后，测试准确度达到了`97.7%`(*add by alex*我的测试结果更好)。对于非常简单的手写数字图片识别任务，古老的`LeNet-5`网络已经可以取得很好的效果，但是稍复杂一点的任务，比如彩色动物图片识别，`LeNet-5`性能就会急剧下降。


## 10.5 表示学习
研究人员发现网络层数越深，模型的表达能力越强，也就越有可能取得更好的性能。那么层层堆叠的卷积网络到底学到了什么特征，使得层数越深，网络的表达能力越强呢？

2014年，`Matthew Zeiler`等人尝试利用可视化的方法去理解卷积神经网络到底学到了什么。通过将每层的特征图利用`反卷积`网络(Deconvolutional Network)映射回输入图片，即可查看学到的特征分布，如`图10.32`所示。可以观察到，第二层的特征对应到边、角、色彩等底层图像提取；第三层开始捕获到纹理这些中层特征；第四、五层呈现了物体的部分特征，如小狗的脸部、鸟类的脚部等高层特征。通过这些可视化的手段，我们可以一定程度上感受卷积神经网络的特征学习过程。

<img src="images/10_32.png" style="width:500px;"/>

图片数据的识别过程一般认为也是`表示学习`(Representation Learning)的过程，从接受到的原始像素特征开始，逐渐提取边缘、角点等底层特征，再到纹理等中层特征，再到头部、物体部件等高层特征，最后的网络层基于这些学习到的抽象特征表示(Representation)做分类逻辑的学习。学习到的特征越高层、越准确，就越有利于分类器的分类，从而获得较好的性能。

从表示学习的角度来理解，卷积神经网络通过层层堆叠来逐层提取特征，网络训练的过程可以看成特征的学习过程，基于学习到的高层抽象特征可以方便地进行分类任务。

应用表示学习的思想，训练好的卷积神经网络往往能够学习到较好的特征，这种特征的提取方法一般是通用的。比如在猫、狗任务上学习到头、脚、身躯等特征的表示，在其它动物上也能够一定程度上使用。基于这种思想，可以将在任务A上训练好的深层神经网络的前面数个特征提取层迁移到任务B上，只需要训练任务B的分类逻辑，即可取得非常好的效果，这种方式是迁移学习的一种，从神经网络角度也称为`网络微调`(Fine-tuning)。

## 10.6 梯度传播
对卷积神经网络的使用有了初步的了解后，现在我们来解决一个关键问题，卷积层通过移动感受野的方式实现离散卷积操作，那么它的梯度传播是怎么进行的呢？

考虑一简单的情形，输入为$3\times3$的单通道矩阵，与一个$2\times2$的卷积核，进行卷积运算，输出结果打平后直接与虚构的标注计算误差，如`图10.33`所示。我们来讨论这种情况下的梯度更新方式。

<img src="images/10_33.png" style="width:500px;"/>

首先推导出输出张量$O$的表达形式：
$$\begin{matrix}
o_{00} = x_{00}w_{00} + x_{01}w_{01} + x_{10}w_{10} + x_{11}w_{11} + b \\
o_{01} = x_{01}w_{00} + x_{02}w_{01} + x_{11}w_{10} + x_{12}w_{11} + b \\
o_{10} = x_{10}w_{00} + x_{11}w_{01} + x_{20}w_{10} + x_{21}w_{11} + b \\
o_{11} = x_{11}w_{00} + x_{12}w_{01} + x_{21}w_{10} + x_{22}w_{11} + b \\
\end{matrix}$$

以$w_{00}$的梯度计算为例，通过链式法则分解：
+ $\displaystyle\frac{\partial\mathcal{L}}{\partial w_{00}} = \sum_{i\in(00,01,10,11)}\frac{\partial\mathcal{L}}{\partial o_j}\frac{\partial o_j}{\partial w_{00}}$

其中，$\displaystyle\frac{\partial\mathcal{L}}{\partial o_i}$可直接由误差函数推导出来，我们直接来考虑$\displaystyle\frac{\partial o_j}{\partial w_i}$，例如：
+ $\displaystyle\frac{\partial o_{00}}{\partial w_{00}} = \frac{\partial(x_{00}w_{00} + x_{01}w_{01} + x_{10}w_{10} + x_{11}w_{11} + b)}{w_{00}} = x_{00}$

同样的方法，可以推导出：
+ $\displaystyle\frac{\partial o_{01}}{\partial w_{00}} = \frac{\partial(x_{01}w_{00} + x_{02}w_{01} + x_{11}w_{10} + x_{12}w_{11} + b)}{w_{00}} = x_{01}$
+ $\displaystyle\frac{\partial o_{01}}{\partial w_{00}} = \frac{\partial(x_{10}w_{00} + x_{11}w_{01} + x_{20}w_{10} + x_{21}w_{11} + b)}{w_{00}} = x_{10}$
+ $\displaystyle\frac{\partial o_{01}}{\partial w_{00}} = \frac{\partial(x_{11}w_{00} + x_{12}w_{01} + x_{21}w_{10} + x_{22}w_{11} + b)}{w_{00}} = x_{11}$

可以观察到，通过循环移动感受野的方式并没有改变网络层可导性，同时梯度的推导也并不复杂，只是当网络层数增大以后，人工梯度推导将变得十分的繁琐。深度学习框架可以自动完成所有参数的梯度计算与更新，我们只需要设计好网络结构即可。

## 10.7 池化层
在卷积层中，可以通过调节步长参数$s$实现特征图的高宽成倍缩小，从而降低了网络的参数量。实际上，除了通过设置步长，还有一种专门的网络层可以实现尺寸缩减功能，即`池化层`(Pooling Layer)。

池化层同样基于局部相关性的思想，通过从局部相关的一组元素中进行采样或信息聚合，从而得到新的元素值：
+ **最大池化层**(Max Pooling)从局部相关元素集中选取最大的一个元素值
+ **平均池化层**(Average Pooling)从局部相关元素集中计算平均值并返回

以$5\times5$输入$X$的最大池化层为例，考虑池化感受野窗口大小$k=2$，步长$s=1$的情况，如`图10.34`所示：

<img src="images/10_34.png" style="width:450px;"/>

循环往复，直至最下方、最右边，获得最大池化层的输出：

<img src="images/10_35.png" style="width:450px;"/>

由于池化层没有需要学习的参数，计算简单，并且可以有效减低特征图的尺寸，非常适合图片这种类型的数据。通过精心设计池化层感受野的高宽$k$和步长$s$参数，可以实现各种降维运算。

## 10.8 BatchNorm层
卷积神经网络的出现，网络参数量大大减低，使得几十层的深层网络成为可能。然而，网络的加深使得网络训练变得非常不稳定，甚至出现网络长时间不更新甚至不收敛的现象，同时网络对超参数比较敏感，超参数的微量扰动也会导致网络的训练轨迹完全改变。

2015年，`Google`研究人员`Sergey Ioffe`等提出了一种`参数标准化`(Normalize)的手段，并基于参数标准化设计了`Batch Nomalization`(简写为`BatchNorm`，或`BN`)层。`BN层`的提出，使得网络的超参数的设定更加自由，比如更大的学习率、更随意的网络初始化等，同时网络的收敛速度更快，性能也更好。`BN层`提出后便广泛地应用在各种深度网络模型上，`卷积层`、`BN层`、`ReLU层`、`池化层`一度成为网络模型的标配单元块，通过堆叠`Conv-BN-ReLU-Pooling`方式往往可以获得不错的模型性能。

为什么需要对网络中的数据进行标准化操作？这个问题很难从理论层面解释透彻，即使是`BN层`的作者给出的解释也未必让所有人信服。与其纠结其缘由，不如通过具体问题来感受数据标准化后的好处。

考虑`Sigmoid`激活函数和它的梯度分布，如`图10.39`所示：

<img src="images/10_39.png" style="width:300px;"/>

`Sigmoid`函数在$x \in [−2,2]$区间的导数值在$[0.1, 0.25]$区间分布；当$x \gt 2$或$x \lt −2$时，`Sigmoid`函数的导数逼近于0，从而容易出现`梯度弥散`现象。为了避免梯度弥散现象，将函数输入$x$标准化映射到0附近的一段较小区间将变得非常重要，可以从`图10.39` 看到。这是使用标准化手段受益的一个例子。

再看一个例子。考虑 2 个输入节点的线性模型，如`图10.40(a)`所示：

<img src="images/10_40.png" style="width:500px;"/>

+ $\mathcal{L} = a = x_1w_1+x_2w_2+b$

讨论如下2种输入分布下的优化问题：
+ 输入$x_1 \in [1, 10]，x_2 \in [1, 10]$
+ 输入$x_1 \in [1, 10]，x_2 \in [100, 1000]$ 

由于模型相对简单，可以绘制出2种$x_1, x_2$下，函数的损失等高线图，`图10.40(b)`是$x_1 \in [1, 10]，x_2 \in [100, 1000]$时的某条优化轨迹线示意，`图10.40(c)`是$x_1 \in [1, 10]，x_2 \in [1, 10]$时的某条优化轨迹线示意，图中的圆环中心即为全局极值点。

考虑到
$$\begin{matrix}
\displaystyle\frac{\partial\mathcal{L}}{\partial w_1} = x_1\\
\displaystyle\frac{\partial\mathcal{L}}{\partial w_2} = x_2\\
\end{matrix}$$

+ 当$x_1, x_2$输入分布相近时，$\frac{\partial\mathcal{L}}{\partial w_1}, \frac{\partial\mathcal{L}}{\partial w_2}$偏导数值相当，函数的优化轨迹如`图10.40(c)`所示
+ 当$x_1, x_2$输入分布差距较大时，某条可能的优化轨迹如`图10.40(b)`所示

对比2条优化轨迹线可以观察到，$x_1, x_2$分布相近时损失函数收敛更加快速，优化轨迹更理想。

通过上述例子我们能够经验性归纳出：网络层输入$x$分布相近，并且分布在较小范围内时(如0附近)，更有利于函数的优化。我们通过数据标准化实现此目的，即将数据$x$映射到$\hat{x}$:
+ $\hat{x} = \displaystyle\frac{x-\mu_r}{\sqrt{\sigma_{r}^2+\epsilon}}$

其中$\mu_r、\sigma_{r}^2$来自统计的所有数据的均值和方差，$\epsilon$是为防止出现除0错误而设置的较小数字，如$1e−8$。

在基于`Batch`的训练阶段，`Batch`内部的均值$\mu_B$和$\sigma_{B}^2$定义如下：
$$\begin{matrix}
\displaystyle\mu_B = \frac{1}{m}\sum_{i=1}^{m}x_i\\
\displaystyle\sigma_{B}^2 = \frac{1}{m}\sum_{i=1}^{m}(x_i-\mu_B)^2\
\end{matrix}$$

其中，$m$为`Batch`中的样本数。

因此，在训练阶段，通过
+ $\hat{x}_{\mathrm{train}} = \displaystyle\frac{x_{\mathrm{train}}-\mu_{B}}{\sqrt{\sigma_{B}^2+\epsilon}}$

标准化输入，并记录每个`Batch`的统计数据$\mu_B$和$\sigma_{B}^2$，用于统计真实的全局$\mu_r$和$\sigma_{r}^2$。

在测试阶段，根据记录的每个`Batch`的$\mu_B$和$\sigma_{B}^2$估计出所有训练数据的$\mu_r$和$\sigma_{r}^2$，按着
+ $\hat{x}_{\mathrm{test}} = \displaystyle\frac{x_{\mathrm{test}}-\mu_{r}}{\sqrt{\sigma_{r}^2+\epsilon}}$

将每层的输入标准化。

上述的标准化运算并没有引入额外的待优化变量，$\mu_r、\sigma_{r}^2$和$\mu_B、\sigma_{B}^2$均由统计得到，不需要参与梯度更新。实际上，为了提高`BN层`的表达能力，`BN层`作者引入了`scale and shift`技巧，将$\hat{x}$变量再次映射变换：
+ $\widetilde{x} = \hat{x}\cdot\lambda+\beta$

其中$\lambda$实现对标准化后的$\hat{x}$再次进行缩放，$\beta$实现对标准化的$\hat{x}$进行平移，不同的是，$\lambda、\beta$均由反向传播算法自动优化，实现网络层`按需`缩放平移数据的分布的目的。

下面我们来学习在`TensorFlow`中实现的`BN层`的方法。

### 10.8.1 前向传播
我们分训练阶段和测试阶段来讨论前向传播过程。

##### 训练阶段
首先计算当前`Batch`的$\mu_B、\sigma_{B}^2$，根据
+ $\widetilde{x}_{\mathrm{train}} = \displaystyle\frac{x_{\mathrm{train}}-\mu_{B}}{\sqrt{\sigma_{B}^2+\epsilon}}\cdot\lambda+\beta$

计算`BN层`的输出。

同时按照
+ $\mu_r \gets \mathrm{momentum}\cdot\mu_r + (1−\mathrm{momentum})\cdot\mu_{B}$
+ $\sigma_{r}^2 \gets \mathrm{momentum}\cdot\sigma_{r}^2 + (1−\mathrm{momentum})\cdot\sigma_{B}^2$

迭代更新全局训练数据的统计值$\mu_r$和$\sigma_{r}^2$，其中$\mathrm{momentum}$是超参数，用于平衡$\mu_r$和$\sigma_{r}^2$的更新幅度：
+ 当$\mathrm{momentum}=0$时，$\mu_r$和$\sigma_{r}^2$直接被设置为最新一个`Batch`的$\mu_B$和$\sigma_{B}^2$
+ 当$\mathrm{momentum}=1$时，$\mu_r$和$\sigma_{r}^2$保持不变，忽略最新一个`Batch`的$\mu_B$和$\sigma_{B}^2$

在`TensorFlow`中，$\mathrm{momentum}$默认设置为0.99。

##### 测试阶段
`BN层`根据
+ $\widetilde{x}_{\mathrm{test}} = \displaystyle\frac{x_{\mathrm{test}}-\mu_{B}}{\sqrt{\sigma_{B}^2+\epsilon}}\cdot\lambda+\beta$

计算输出$\widetilde{x}_{\mathrm{test}}$，其中$\mu_r、\sigma_{r}^2、\lambda、\beta$均来自训练阶段统计或优化的结果，在测试阶段直接使用，并不会更新这些参数。

### 10.8.2 反向更新
在训练模式下的反向更新阶段，反向传播算法根据损失$\mathcal{L}$求解梯度$\frac{\partial\mathcal{L}}{\partial\lambda}$和$\frac{\partial\mathcal{L}}{\partial\beta}$，并按着梯度更新法则自动优化$\lambda、\beta$参数。

需要注意的是，对于2D特征图输入$X:[b,h,w,c]$，`BN层`并不是计算每个点的$\mu_B、\sigma_{B}^2$，而是在通道轴$c$上面统计每个通道上面所有数据的$\mu_B、\sigma_{B}^2$，因此$\mu_B、\sigma_{B}^2$是每个通道上所有其它维度的均值和方差。以`shape`为$[100,32,32,3]$的输入为例，在通道轴$c$上面的均值计算如下：

In [31]:
# 构造输入
x=tf.random.normal([100,32,32,3])
# 将其他维度合并，仅保留通道维度
x=tf.reshape(x,[-1,3])
# 计算其他维度的均值
ub=tf.reduce_mean(x,axis=0)
x.shape, ub

(TensorShape([102400, 3]),
 <tf.Tensor: shape=(3,), dtype=float32, numpy=array([ 0.00345077,  0.0024313 , -0.00177814], dtype=float32)>)

数据有$c$个通道数，因此有$c$个均值产生。

除了在$c$轴上面统计数据$\mu_B、\sigma_{B}^2$的方式，我们也很容易将其推广至其它维度计算均值的方式，如`图10.41`所示：

<img src="images/10_41.png" style="width:500px;"/>

+ **Layer Norm**：统计每个样本的所有特征的均值和方差
+ **Instance Norm**：统计每个样本的每个通道上特征的均值和方差
+ **Group Norm**：将$c$通道分成若干组，统计每个样本的通道组内的特征均值和方差

上面提到的`Normalization`方法均由独立的几篇论文提出，并在某些应用上验证了其相当或者优于`BatchNorm`算法的效果。由此可见，深度学习算法研究并非难于上青天，只要多思考、多锻炼算法工程能力，人人都有机会发表创新性成果。

### 10.8.3 BN层实现
`TensorFlow`中通过`layers.BatchNormalization()`类可以非常方便地实现`BN层`：

```python
# 创建BN层
# BN层的训练阶段和测试阶段的行为不同，需要通过设置training标志位来区分训练模式还是测试模式
layer=layers.BatchNormalization()
```

以`LeNet-5`的网络模型为例，在卷积层后添加`BN层`：

```python
network = Sequential([
    layers.Conv2D(6,kernel_size=3,strides=1),
    # 插入BN层
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=2,strides=2),
    layers.ReLU(),
    layers.Conv2D(16,kernel_size=3,strides=1),
    # 插入BN层
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=2,strides=2),
    layers.ReLU(),
    layers.Flatten(),
    layers.Dense(120, activation='relu'),
    # 此处也可以插入BN层
    layers.Dense(84, activation='relu'),
    # 此处也可以插入BN层
    layers.Dense(10)
])
```

在训练阶段，需要设置网络的参数`training=True`以区分`BN层`是训练还是测试模型：

```python
with tf.GradientTape() as tape:
    # 插入通道维度
    x = tf.expand_dims(x,axis=3)
    # 前向计算，设置计算模式，[b, 784] => [b, 10]
    out = network(x, training=True)
```

在测试阶段，需要设置`training=False`，避免`BN层`采用错误的行为：

```python
for x,y in db_test: # 遍历测试集
    # 插入通道维度
    x = tf.expand_dims(x,axis=3)
    # 前向计算，测试模式
    out = network(x, training=False)
```

## 10.9 经典卷积网络

### 10.9.1 AlexNet
2012年，`ILSVRC12`挑战赛`ImageNet`数据集分类任务的冠军`Alex Krizhevsky`提出了8层的深度神经网络模型`AlexNet`，它接收输入为$22\times22$大小的彩色图片数据，经过五个卷积层和三个全连接层后得到样本属于1000个类别的概率分布。为了降低特征图的维度，`AlexNet`在第1、2、5个卷积层后添加了`MaxPooling`层，如`图10.43`所示：

<img src="images/10_43.png" style="width:600px;"/>

网络的参数量达到了6000万个。为了能够在当时的显卡设备`NVIDIA GTX 580`(3GB显存)上训练模型，`Alex Krizhevsky`将卷积层、前2个全连接层等拆开在两块显卡上面分别训练，最后一层合并到一张显卡上面，进行反向传播更新。`AlexNet`在`ImageNet`取得了`15.3%`的Top5错误率，比第二名在错误率上降低了`10.9%`。

`AlexNet`的创新之处在于：
+ 层数达到了较深的8层
+ 采用了`ReLU`激活函数，过去的神经网络大多采用`Sigmoid`激活函数，计算相对复杂，容易出现梯度弥散现象
+ 引入`Dropout`层，提高了模型的泛化能力，防止过拟合

### 10.9.2 VGG系列
2014年，`ILSVRC14`挑战赛`ImageNet`分类任务的亚军牛津大学`VGG`实验室提出了`VGG11、 VGG13、VGG16、VGG19`等一系列的网络模型。

以`VGG16`为例，它接受$22\times22$大小的彩色图片数据，经过2个`Conv-Conv-Pooling`单元，和3个`Conv-Conv-Conv-Pooling`单元的堆叠，最后通过3层全连接层输出当前图片分别属于1000类别的概率分布，如`图10.44`所示。`VGG16`在`ImageNet`取得了`7.4%`的Top5错误率，比`AlexNet`在错误率上降低了`7.9%`。

<img src="images/10_44.png" style="width:600px;"/>

`VGG`系列网络的创新之处在于：
+ 层数提升至19层
+ 全部采用更小的$3\times3$卷积核，相对于`AlexNet`中的$7\times7$卷积核，参数量更少，计算代价更低
+ 采用更小的池化层$2\times2$窗口和步长$s = 2$，而`AlexNet`中是步长$s=2$、$3\times3$的池化窗口

### 10.9.3 GoogLeNet
$3\times3$的卷积核参数量更少，计算代价更低，同时在性能表现上甚至更优越，因此业界开始探索卷积核最小的情况：$1\times1$卷积核。如`图10.46`所示，输入为3通道的$5\times5$图片，与单个$1\times1$的卷积核进行卷积运算，每个通道的数据与对应通道的卷积核运算，得到3个通道的中间矩阵，对应位置相加得到最终的输出张量。

<img src="images/10_46.png" style="width:400px;"/>

对于输入`shape`为$[b,h,w,c_{in}]$，$1\times1$卷积层的输出为$[b,h,w,c_{out}]$，其中$c_{in}$为输入数据的通道数，$c_{out}$为输出数据的通道数，也是$1\times1$卷积核的数量。$1\times1$卷积核的一个特别之处在于，它可以不改变特征图的宽高，而只对通道数$c$进行变换。

2014年，`ILSVRC`挑战赛的冠军`Google`提出了大量采用$3\times3$和$1\times1$卷积核的网络模型：`GoogLeNet`，网络层数达到了22层。虽然`GoogLeNet`的层数远大于`AlexNet`，但是它的参数量却只有`AlexNet`的$\frac{1}{12}$，同时性能也远好于`AlexNet`。在`ImageNet`数据集分类任务上取得了`6.7%`的Top5错误率，比`VGG16`在错误率上降低了`0.7%`。

`GoogLeNet`网络采用模块化设计的思想，通过大量堆叠`Inception`模块，形成了复杂的网络结构，如`图10.47`所示：

<img src="images/10_47.png" style="width:300px;"/>

`Inception`模块的输入为$X$，通过4个子网络得到4个网络 输出，在通道轴上面进行拼接合并，形成`Inception`模块的输出。这4个子网络是：
+ $1\times1$卷积层
+ $1\times1$卷积层，再通过一个$3\times3$卷积层
+ $1\times1$卷积层，再通过一个$5\times5$卷积层
+ $3\times3$最大池化层，再通过$1\times1$卷积层

`GoogLeNet`的网络结构如`图10.48`所示，其中红色框中的网络结构即为`图10.47`的网络结构。

<img src="images/10_48.png" style="width:800px;"/>